In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from functools import partial
import itertools
import os
import json
from decimal import Decimal
import time
from scipy.integrate import solve_ivp
import pandas as pd
import traceback
import copy
import concurrent.futures
import multiprocessing as mp
import sys
import uuid
import matplotlib.patheffects as pe
import shutil
import seaborn as sns
import gc

from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams.update({'font.size': 13})

In [ ]:
def sci_notation(number, sig_fig=2):
    form_string = "{0:.{1:d}e}".format(number, sig_fig)
    a,b = form_string.split("e")
    b = int(b)
    c = r"$10^{"+str(b)+r"}$"
    if float(number).is_integer() and number <= 10:
        return str(int(number))
    elif a == '1.00':
        return c
    elif float(a).is_integer():
        return str(int(float(a))) + r"$\cdot$" + c
    elif b == 0:
        return a
    else:
        return a + r"$\cdot$" + c

In [ ]:
def axes_tick_formatter(self, MajorTicks, MinorTicks, axis = 'x'):
    if axis == 'x':
        self.xaxis.set_major_locator(MultipleLocator(MajorTicks))
        self.xaxis.set_minor_locator(MultipleLocator(MinorTicks))
    elif axis == 'y':
        self.yaxis.set_major_locator(MultipleLocator(MajorTicks))
        self.yaxis.set_minor_locator(MultipleLocator(MinorTicks))
    else:
        print("Algo mal hay")

In [ ]:
def remove_zeros(a):
    
    if a[-1] != '0' and a[-1] != '.':
        return str(a)   
        
    else:
        c = str(a[:-1])
        return(remove_zeros(c))

def param_folder_notation(number, sig_fig=2):
    if number == 0.0:
        c = '0'
    
    else:    
        form_string = "{0:.{1:d}e}".format(number, sig_fig)
        a,b = form_string.split("e")
        a = remove_zeros(a)
        b = int(b)

        if float(a).is_integer():
            if b == 0:
                c = str(int(float(a)))
            else:
                c = str(int(float(a))) +'e'+ str(b)
        else:
            if b == 0:
                c = a
            else:
                c = a +'e'+ str(b)
        c = c.replace('-','m').replace('.','p')
    
    return c

In [ ]:
def m_notch_epsin_coop(t, x, params):
    bl = params["bl"]
    dl = params["dl"]
    
    kr = params["kr"]
    krl = params["krl"]
    
    br = params["br"]
    dr = params["dr"]
    
    kb = params["kb"]
    ku = params["ku"]
    
    km = params["km"]
    n = params["n"]
    m = params["m"]
    
    
    ks2 = params["ks2"]
    kp = params["kp"]
    
    ki = params["ki"]
    ks3 = params["ks3"]
    ds = params["ds"]
    
    KG = params["KG"]
    TN = params["TN"]
    CN = params["CN"]
    
    dG = params["dG"]
    dm = params["dm"]
    
    R = x[0]
    L = x[1]
    E = x[2]
    C = x[3]
    CL = x[4]
    CLi = x[5]
    CR = x[6]
    S = x[7]
    G4 = x[8]
    Cit = x[9]

    
    Dx = np.array([
        br - dr * R - kb * (R * L) + ku * C, #R
        bl - dl * L - kb * (R * L) + ku * C + n * krl * kr * CLi , #L
        m * kr * CLi - m * ks2 * km ** (1-m) * C ** n * E ** m , #E
#         0., #E
        kb * R * L - ku * C - kp * C - n * ks2 * km ** (1-m) * C ** n * E ** m, #C
        ks2 * km ** (1-m) * C ** n * E ** m - ki * CL, #CL
        ki * CL - kr * CLi, #CLi
        ks2 * km ** (1-m) * C ** n * E ** m - ks3 * CR, #CR
        n * ks3 * CR - ds * S, #S
        n * ks3 * CR - dG * G4, #G4
        CN * TN * G4 / (G4 + KG) - dm * Cit #Cit
        
    ])
    
    return Dx

In [ ]:
def m_gillespie_notch_epsin_coop(t, x, params):

    bl = params["bl"]
    dl = params["dl"]

    kr = params["kr"]
    krl = params["krl"]

    br = params["br"]
    dr = params["dr"]

    kb = params["kb"]
    ku = params["ku"]

    km = params["km"]
    n = params["n"]
    m = params["m"]


    ks2 = params["ks2"]
    kp = params["kp"]

    ki = params["ki"]
    ks3 = params["ks3"]
    ds = params["ds"]

    t12_G = params["t12_G"]
    t12_m = params["t12_m"]

    KG = params["KG"]
    TN = params["TN"]
    CN = params["CN"]

    dG = np.log(2) * TN / t12_G
    dm = np.log(2) * TN / t12_m


    R = x[0]
    L = x[1]
    E = x[2]
    C = x[3]
    CL = x[4]
    CLi = x[5]
    CR = x[6]
    S = x[7]
    G4 = x[8]
    Cit = x[9]

    Em = np.prod([E - i for i in range(m)], dtype='float64')
    Cn = np.prod([C - i for i in range(n)], dtype='float64')
        
    if C < 0 or E < 0:
        print([C,E])
    # Aquí defino las reacciones que pueden suceder
    rates = np.array([
        br , #0
        dr * R , #1
        bl , #2
        dl * L , #3
        kb * (R * L) , #4
        ku * C, #5
        ks2 * km ** (1-m) * Em * Cn, #6
        kr * CLi,#7 
        kp * C, #8
        ki * CL, #9
        ks3 * CR, #10
        ds * S, #11
        dG * G4, #12
        CN * TN * G4 / (G4 + KG) , #13
        dm * Cit]) #14
        
    rates_sum = np.sum(rates)
    
    try:
    
        if rates_sum != 0:
            #Defino el tiempo de la siguiente reacción
            tau = np.random.exponential(scale = 1/rates_sum)
            T = t + tau

            #Numero aleatorio para definir la siguiente reacción
            ran = np.random.uniform(0,1)

    #         Decido la siguiente reacción: me quedo con la que tien el índice más bajo tal que 
    #         la suma de las probabilidades de todas las reacciones hasta cada reacción sea mayor que
    #         la suma de las probabilidades de todas las reacciones multiplicado por el número aleatorio

            reaction = min([i for i in range(len(rates)) if sum(rates[:i+1]) >= ran * rates_sum])

            y = list(x)

    #         Según el índice que he obtenido realizo una de las reacciones

            if reaction == 0:
                y[0] += 1
    #             br , #0

            if reaction == 1:
                y[0] -= 1
    #             dr * R , #1

            if reaction == 2:
                y[1] += 1
    #             bl , #2


            if reaction == 3:
                y[1] -= 1
    #         dl * L , #3

            if reaction == 4:
                y[0] -= 1
                y[1] -= 1
                y[3] += 1
    #             kb * (R * L) , #4

            if reaction == 5:
                y[0] += 1
                y[1] += 1
                y[3] -= 1    
    #             ku * C, #5

            if reaction == 6:
                y[2] -= m
                y[3] -= n
                y[4] += 1
                y[6] += 1
    #             ks2 * km ** (1-m) * C ** n * E ** m , #6

            if reaction == 7:
                y[2] += m
                y[5] -= 1
    #             kr * CLi,#7 

            if reaction == 8:
                y[3] -= 1
    #             kp * C, #8

            if reaction == 9:
                y[4] -= 1
                y[5] += 1
    #             ki * CL, #9

            if reaction == 10:
                y[6] -= 1
                y[7] += n
                y[8] += n
    #             ks3 * CR, #10

            if reaction == 11:
                y[7] -= 1
    #             ds * S, #11

            if reaction == 12:
                y[8] -= 1
    #             dG * G4, #12

            if reaction == 13:
#                 y[8] -= 1
                y[9] += 1
                
    #             G4 / (G4 + KG) , #13

            if reaction == 14:
                y[9] -= 1
    #             dm * Cit #14

            return T, y
        else:
            return -1
    except Exception:
        print(rates_sum)
        print(rates)
        print(x)
        print(traceback.format_exc())

In [ ]:
def x_interpolation(r, q, y):
    if r[0] >= q[0]:
        p_0 = q
        p_1 = r
    else:
        p_0 = r
        p_1 = q

    x = p_0[0] + (y - p_0[1]) / (p_1[1] - p_0[1]) * (p_1[0] - p_0[0])

    return x


def peak_values (np_input_x, np_input_y, eps = 0.1):
    max_value = max(np_input_y)
    max_value_list = np.where(np_input_y == max_value)
    max_value_index = int(np.round(np.mean(max_value_list[0])))

    end_value = np_input_y[-1]

    t2 = 0
    t1 = 0

    if end_value < 0.5 * max_value:

        half_max = 0.5 * max_value

        mvi_left = max_value_index - 1
        mvi_right = max_value_index + 1

        while np_input_y[mvi_left] > half_max:
            mvi_left -= 1

        while np_input_y[mvi_right] > half_max:
            mvi_right += 1

        r1 = [np_input_x[mvi_left], np_input_y[mvi_left]]
        q1 = [np_input_x[mvi_left + 1], np_input_y[mvi_left + 1]]

        t1 = x_interpolation(r1, q1, half_max)

        r2 = [np_input_x[mvi_right], np_input_y[mvi_right]]
        q2 = [np_input_x[mvi_right - 1], np_input_y[mvi_right - 1]]

        t2 = x_interpolation(r2, q2, half_max)
        
    max_12 = 0.5 * max_value
    
    k = 0
    
    while np_input_y[k] < max_12:
        k += 1
        
    r12 = [np_input_x[k], np_input_y[k]]
    q12 = [np_input_x[k - 1], np_input_y[k - 1]]
    
    t12 = x_interpolation(r12, q12, max_12)
    
    t_max = np_input_x[max_value_index]

    return (t1, t2, max_value, end_value, t12, t_max)

def HS_f (a,b,eps):
    if a/b>eps:
        return 1.
    else:
        return 0.

In [ ]:
def rk4(f,x,t,dt,params):
    
    k1 = f(t, x, params)
    k2 = f(t, x + k1*dt/2, params)
    k3 = f(t, x + k2*dt/2, params)    
    k4 = f(t, x + k3*dt, params)
    
    k = dt * (k1 + 2*k2 + 2*k3 + k4)/6
    
    return (x + k)

In [ ]:
def par_solve_ivp(model, time_l, x_0, meth, arg):
    return solve_ivp(model, time_l, x_0, method = meth, args=arg)

In [ ]:

llss = ["-", ":", "-.", "--"]

variables = ["R","L","E","C",r"$C_L$",r"$C_{Li}$",r"$C_R$","S","Gal4","Cit"]
variables_f = ["R","L","E","C",r"C_L",r"C_Li",r"C_R","S","Gal4","Cit"]
ligands = ["Dll1", r"$Dll4_{ECD}$"+"\n"+r"$Dll1_{ICD}$", "Dll4", r"$Dll1_{ECD}$"+"\n"+r"$Dll4_{ICD}$"]
ligands_f = ["Dll1", "1I4E", "Dll4", "4I1E"]
cm_variables = ["#194569","#800020","#ff7f0e","#9467bd","#c20e35","#fa8fad","#1f77b4","#80bce6","#c8f1c8","#77dd77"]
cm_ligands = ["#194569","#ff7f0e","#800020","#9467bd"]
cm_ligands_stoch = ["#81bde7","#ffd1ab","#f98eac","#c3abdb"]
variables_pq = ["R","L","E","C","CL","CLi","CR","S","G4","Cit"]


In [ ]:
t12_G = 3.9
t12_m = 3.4

t0 = 0.
tf = 25. * 60.

params_base = {"bl": 2e0, 
    "br": 8e0, 
    "dl": 2e-3, 
    "dr": 2e-3,
    "kb": 2e-4,
    "ku": 2e2, 
    "kp": 5.,
    "ks2": 1.,
    "kr": 1e-4,
    'krl':0.,
    'km':1e3,
    "ki": 1.5e1, 
    "ks3": 1e-1, 
    "ds": 8e-3,
    "n":1,
    "m":1,
    "KG": 6.6 * 0.1 * 3500., 
    "t12_G": t12_G,
    "t12_m": t12_m,
    "dG": np.log(2) / (60. * t12_G),
    "dm": np.log(2) / (60. * t12_m),
    "E0": 3e3,
    "TN": 1/60.,
    "CN": 1000.,
    "t0": t0,
    "tf": tf} 


time_l = [t0,tf]

c_max_11 = 0.68
c_max_41 = 0.59

c_max_44 = 1.4
c_max_14 = 3.6


params_D1 = dict(params_base)
params_D1.update({"kr":1e-4,"n":4, "A":6, "ks2":2e-5, "kb": 1e-3, 'kp': 5, "bl": 2e0, "LT":ligands_f[0]})
params_D1.update({"m":params_D1["A"] * params_D1["n"]})

params_1I_4E = dict(params_base)
params_1I_4E.update({"kr":1e-4,"n":4, "A":1, "ks2":3.0e-4, "kb": 1e-2, 'kp': 5, "bl": 6e0, "LT":ligands_f[1]})
params_1I_4E.update({"m":params_1I_4E["A"] * params_1I_4E["n"]})

params_D4 = dict(params_base)
params_D4.update({"kr":2.0e-3,"n":1, "A":1, "ks2":3.0e-4, "kb": 1e-2, 'kp': 5, "bl": 6e0, "LT":ligands_f[2]})
params_D4.update({"m":params_D4["A"] * params_D4["n"]})

params_4I_1E = dict(params_base)
params_4I_1E.update({"kr":2.0e-3,"n":1, "A":6, "ks2":2e-5, "kb": 1e-3, 'kp': 5, "bl": 2e0, "LT":ligands_f[3]})
params_4I_1E.update({"m":params_4I_1E["A"] * params_4I_1E["n"]})

filename0='Fig5'
filename1='S15_S16'

In [ ]:
figures_path = os.getcwd() + '/' + filename0
if os.path.isdir(figures_path) == False:
    os.makedirs(figures_path)

src_data_file_path = os.getcwd() + '/' + filename1
if os.path.isdir(src_data_file_path) == False:
    os.makedirs(src_data_file_path)

dst_data_file_path = figures_path +'/Data_Files/'
if os.path.isdir(dst_data_file_path) == False:
    os.makedirs(dst_data_file_path)


# S15 S16

In [ ]:

param_iter = []
list_dict_hybrids = []

params = dict(params_D1)
list_dict_hybrids.append(params)

params = dict(params_1I_4E)
list_dict_hybrids.append(params)

params = dict(params_D4)
list_dict_hybrids.append(params)

params = dict(params_4I_1E)
list_dict_hybrids.append(params)

param_iter = []
param_iter_det = []

num_sims = 20

for i in range(len(list_dict_hybrids)):
    params = dict(list_dict_hybrids[i])
    
    X_0 = [round(params["br"]/params["dr"],2), 
            round(params["bl"]/params["dl"],2),
            round(params["E0"],2),
            0., 0., 0., 0., 0., 0., 0.]
    
    params.update({'x0': X_0})
    param_iter_det.append(tuple([dict(params)]))
    for n in range(num_sims):
        params_i = dict(params)
        params_i.update({'seed': n, 'filename': str(params["LT"]) + '_' + str(n) + '.parquet'})
        param_iter.append(params_i)


# CI <> 0
x_0 = [[round(e[0]["br"]/e[0]["dr"],2), 
       round(e[0]["bl"]/e[0]["dl"],2),
       round(e[0]["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in param_iter_det]


In [ ]:
def partial_iter_mp(params):
    filename  = params['filename']
    x_0 = params['x0']
    np.random.seed(params['seed'])
    tf = params['tf']
    Y = []
    Y.append(x_0)
    T = []
    T.append( params['t0'])


    
    while T[-1] <= tf:
        current_step = m_gillespie_notch_epsin_coop(T[-1], Y[-1], params)
        if current_step != -1:
            t_i, y_i = current_step
            Y.append(y_i)
            T.append(t_i)

        else:
            break
    
    sol = np.transpose(Y)
    df = pd.DataFrame({"t": T ,
        "R": sol[0],
        "L": sol[1],
        "E": sol[2],
        "C": sol[3],
        "CL": sol[4],
        "CLi": sol[5],
        "CR": sol[6],
        "S": sol[7],
        "G4": sol[8],
        "Cit": sol[9]})
        
    df.to_parquet(filename)


In [ ]:
os.chdir(src_data_file_path)
with mp.Pool() as pool:
    pool.map(partial_iter_mp, param_iter)
    

In [ ]:
partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)
sol_det = [partial_iter(f,'Radau',e) for e,f in zip(param_iter_det,x_0)]

T_H_det = [e.t/60. for e in sol_det]

In [ ]:
os.chdir(src_data_file_path)

In [ ]:
for i in range(len(list_dict_hybrids)):
    for v in range(len(sol_det[0].y)):
        fig,ax = plt.subplots()
        for j in range(num_sims):
            data_file_name = ligands_f[i] + '_' + str(j) + '.parquet'
            df = pd.read_parquet(dst_data_file_path + data_file_name)
            ax.plot(df["t"]/60., df[variables_pq[v]], color = cm_ligands_stoch[i],zorder=1, linewidth=1, alpha = .4)

        ax.yaxis.set_minor_locator(AutoMinorLocator(n=2))
        axes_tick_formatter(ax, 5, 2.5)
        ax.plot(T_H_det[i], sol_det[i].y[v], color = cm_ligands[i], alpha = 1, label = ligands[i],zorder=2,
                path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()]
               )
        ax.set_ylabel(variables[v] + " [a.u.]")
        ax.set_xlabel(r"$t$ [h]")
        ax.legend()
        fig.savefig(ligands_f[i] + '_' + variables_f[v] + ".png", bbox_inches="tight", dpi = 300, transparent = True)
        
        plt.close()
        
        del (fig, df)
        gc.collect()

# Fig 5

# Panels A, B, C & D (Data From S15-S16)

In [ ]:

fig,ax = plt.subplots()
i = 0
v = 9
for j in range(num_sims):
    data_file_name = ligands_f[i] + '_' + str(j) + '.parquet'
    
    if os.path.isfile(dst_data_file_path + data_file_name) == False:
        shutil.copyfile(src_data_file_path + data_file_name, dst_data_file_path + data_file_name)
        
    df = pd.read_parquet(dst_data_file_path + data_file_name)
    # ax.plot(df["t"]/60., df[variables_pq[v]], color = cm_ligands_stoch[i],zorder=1, linewidth=.5)
    ax.plot(df["t"]/60., df[variables_pq[v]], color = cm_ligands_stoch[i],zorder=1, linewidth=1, alpha = .4)

ax.plot(T_H_det[i], sol_det[i].y[v], color = cm_ligands[i], alpha = 1, label = ligands[i],zorder=2,
                path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()]
       )
ax.set_ylabel(variables[v] + " [a.u.]")
ax.set_xlabel(r"$t$ [h]")
ax.legend()

pvs = peak_values(T_H_det[i], sol_det[i].y[v])
ax.arrow(pvs[0]*1.18, pvs[2]/2,(pvs[1]-pvs[0])*.945,0, head_width=5, head_length=0.4, color=cm_variables[2], zorder = 1)
ax.arrow(pvs[1]*0.97, pvs[2]/2,(pvs[0]-pvs[1])*.93,0, head_width=5, head_length=0.4, color=cm_variables[2], zorder = 1)
ax.text(0.2 * (pvs[1]-pvs[0]),1.05 * pvs[2]/2,r"FWHM $\approx$" + str(round(pvs[1]-pvs[0], 2)) + " h")
ax.text(0.2 * (pvs[1]-pvs[0]),0.7 * pvs[2]/2,r"Experimental FWHM:"+ "\n" + r"95% IC= [12,13.3] h")

axes_tick_formatter(ax, 50, 25, axis = 'y')
axes_tick_formatter(ax, 5, 2.5)

fig.savefig(figures_path + '/' + filename0 + "_A.png", bbox_inches="tight", dpi = 300, transparent = True)

del (fig, df)
gc.collect()

In [ ]:

fig,ax = plt.subplots()
i = 1
v = 9
for j in range(num_sims):
    data_file_name = ligands_f[i] + '_' + str(j) + '.parquet'
    
    if os.path.isfile(dst_data_file_path + data_file_name) == False:
        shutil.copyfile(src_data_file_path + data_file_name, dst_data_file_path + data_file_name)
        
    df = pd.read_parquet(dst_data_file_path + data_file_name)
    ax.plot(df["t"]/60., df[variables_pq[v]], color = cm_ligands_stoch[i],zorder=1, linewidth=1, alpha = .4)

ax.plot(T_H_det[i], sol_det[i].y[v], color = cm_ligands[i], alpha = 1, label = ligands[i],zorder=2,
                path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()]
       )
ax.set_ylabel(variables[v] + " [a.u.]")
ax.set_xlabel(r"$t$ [h]")
ax.legend()

pvs = peak_values(T_H_det[i], sol_det[i].y[v])
ax.arrow(pvs[0]*1.18, pvs[2]/2,(pvs[1]-pvs[0])*.945,0, head_width=20, head_length=0.4, color=cm_variables[0], zorder = 1)
ax.arrow(pvs[1]*0.97, pvs[2]/2,(pvs[0]-pvs[1])*.93,0, head_width=20, head_length=0.4, color=cm_variables[0], zorder = 1)
ax.text(0.23 * (pvs[1]-pvs[0]),1.05 * pvs[2]/2,r"FWHM$\approx$" + str(round(pvs[1]-pvs[0], 2)) + " h")
ax.text(0.12 * (pvs[1]-pvs[0]),0.84 * pvs[2]/2,r"Experimental FWHM$\approx$15.63 h")

axes_tick_formatter(ax, 200, 100, axis = 'y')
axes_tick_formatter(ax, 5, 2.5)

fig.savefig(figures_path + '/' + filename0 + "_I.png", bbox_inches="tight", dpi = 300, transparent = True)

del (fig, df)
gc.collect()

In [ ]:

fig,ax = plt.subplots()
i = 2
v = 9
for j in range(num_sims):
    data_file_name = ligands_f[i] + '_' + str(j) + '.parquet'
    
    if os.path.isfile(dst_data_file_path + data_file_name) == False:
        shutil.copyfile(src_data_file_path + data_file_name, dst_data_file_path + data_file_name)
        
    df = pd.read_parquet(dst_data_file_path + data_file_name)
    ax.plot(df["t"]/60., df[variables_pq[v]], color = cm_ligands_stoch[i],zorder=1, linewidth=1, alpha = .4)

ax.plot(T_H_det[i], sol_det[i].y[v], color = cm_ligands[i], alpha = 1, label = ligands[i],zorder=2,
                path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()]
       )
ax.set_ylabel(variables[v] + " [a.u.]")
ax.set_xlabel(r"$t$ [h]")
ax.legend()

axes_tick_formatter(ax, 100, 50, axis = 'y')
axes_tick_formatter(ax, 5, 2.5)

fig.savefig(figures_path + '/' + filename0 + "_C.png", bbox_inches="tight", dpi = 300, transparent = True)

del (fig, df)
gc.collect()

In [ ]:

fig,ax = plt.subplots()
i = 3
v = 9
for j in range(num_sims):
    data_file_name = ligands_f[i] + '_' + str(j) + '.parquet'
    
    if os.path.isfile(dst_data_file_path + data_file_name) == False:
        shutil.copyfile(src_data_file_path + data_file_name, dst_data_file_path + data_file_name)
        
    df = pd.read_parquet(dst_data_file_path + data_file_name)
    ax.plot(df["t"]/60., df[variables_pq[v]], color = cm_ligands_stoch[i],zorder=1, linewidth=1, alpha = .4)

ax.plot(T_H_det[i], sol_det[i].y[v], color = cm_ligands[i], alpha = 1, label = ligands[i],zorder=2,
                path_effects=[pe.Stroke(linewidth=2, foreground='w'), pe.Normal()]
       )
ax.set_ylabel(variables[v] + " [a.u.]")
ax.set_xlabel(r"$t$ [h]")
ax.legend()

axes_tick_formatter(ax, 50, 25, axis = 'y')
axes_tick_formatter(ax, 5, 2.5)

fig.savefig(figures_path + '/' + filename0 + "_G.png", bbox_inches="tight", dpi = 300, transparent = True)

del (fig, df)
gc.collect()

# Panels B, D, H & J

In [ ]:
os.chdir(dst_data_file_path)

In [ ]:

list_dict_hybrids = []

params = dict(params_D1)
params.update({'bl':1})
list_dict_hybrids.append(params)
params = dict(params_D1)
params.update({'bl':2})
list_dict_hybrids.append(params)
params = dict(params_D1)
params.update({'bl':4})
list_dict_hybrids.append(params)
params = dict(params_D1)
params.update({'bl':6})
list_dict_hybrids.append(params)
params = dict(params_D1)
params.update({'bl':8})
list_dict_hybrids.append(params)

params = dict(params_D4)
params.update({'bl':1})
list_dict_hybrids.append(params)
params = dict(params_D4)
params.update({'bl':2})
list_dict_hybrids.append(params)
params = dict(params_D4)
params.update({'bl':4})
list_dict_hybrids.append(params)
params = dict(params_D4)
params.update({'bl':6})
list_dict_hybrids.append(params)
params = dict(params_D4)
params.update({'bl':8})
list_dict_hybrids.append(params)

params = dict(params_1I_4E)
list_dict_hybrids.append(params)

params = dict(params_4I_1E)
list_dict_hybrids.append(params)

param_iter = []
param_iter_det = []

num_sims = 100

for i in range(len(list_dict_hybrids)):
    params = dict(list_dict_hybrids[i])
    
    X_0 = [round(params["br"]/params["dr"],2), 
            round(params["bl"]/params["dl"],2),
            round(params["E0"],2),
            0., 0., 0., 0., 0., 0., 0.]
    
    params.update({'x0': X_0})
    param_iter_det.append(tuple([dict(params)]))
    for n in range(num_sims):
        params_i = dict(params)
        params_i.update({'seed': n})
        param_iter.append(params_i)


In [ ]:
def partial_iter_mp(params):
    x_0 = params['x0']
    np.random.seed(params['seed'])
    tf = params['tf']
    Y = []
    Y.append(x_0)
    T = []
    T.append( params['t0'])

    while T[-1] <= tf:
        current_step = m_gillespie_notch_epsin_coop(T[-1], Y[-1], params)
        if current_step != -1:
            t_i, y_i = current_step
            Y.append(y_i)
            T.append(t_i)

        else:
            break
    
    sol = np.transpose(Y)
    return max(sol[9])

with mp.Pool() as pool:
    sol = pool.map(partial_iter_mp, param_iter)
    

In [ ]:

dict_results = dict({})
for i in range(len(list_dict_hybrids)):
    listname = list_dict_hybrids[i]['LT'] + "_" + str(list_dict_hybrids[i]['bl'])
    listvalues = [sol[j] for j in range(i*num_sims,(i+1)*num_sims)]
    dict_results.update({listname:listvalues})

df = pd.DataFrame(dict_results)

filename = 'CitMax'
df.to_parquet(filename + '.parquet')


In [ ]:

df = pd.read_parquet(dst_data_file_path+'CitMax.parquet')
df


In [ ]:

df_d1 = df[['Dll1_1','Dll1_2','Dll1_4','Dll1_6','Dll1_8']].copy()
df_d4 = df[['Dll4_1','Dll4_2','Dll4_4','Dll4_6','Dll4_8']].copy()

df_d1 = df_d1.rename(columns={'Dll1_1':'1', 'Dll1_2':'2','Dll1_4':'4','Dll1_6':'6','Dll1_8':'8'})
df_d4 = df_d4.rename(columns={'Dll4_1':'1','Dll4_2':'2','Dll4_4':'4','Dll4_6':'6','Dll4_8':'8'})


In [ ]:

fig, ax1 = plt.subplots()

sns.set(style="whitegrid")
tips = sns.load_dataset("tips")

cpalette = [cm_variables[0] for i in range(4)]

sns.boxplot(data=df_d1, showfliers = False, ax = ax1, boxprops={'facecolor':'None'})
sns.swarmplot(data=df_d1, dodge = 0,size = 2.4, palette=cpalette, ax = ax1, zorder=0)

ax1.set_xlabel(r'$\beta_L\ [min^{-1}]$')
ax1.set_ylabel(r"Cit$_{max}$ [a.u.]")
ax1.set_ylim([70,800])
plt.show()

fig.savefig(figures_path + '/' + filename0 + '_B.png', bbox_inches="tight", dpi = 300, transparent = True)


In [ ]:

fig, ax1 = plt.subplots()

sns.set(style="whitegrid")
tips = sns.load_dataset("tips")

cpalette = [cm_variables[1] for i in range(4)]

sns.boxplot(data=df_d4, showfliers = False, ax = ax1, boxprops={'facecolor':'None'})
sns.swarmplot(data=df_d4, dodge = 0,size = 2.4, palette=cpalette, ax = ax1, zorder=0)

ax1.set_xlabel(r'$\beta_L\ [min^{-1}]$')
ax1.set_ylabel(r"Cit$_{max}$ [a.u.]")

ax1.set_ylim([70,800])
plt.show()

fig.savefig(figures_path + '/' + filename0 + '_D.png', bbox_inches="tight", dpi = 300, transparent = True)


In [ ]:
	
df_ligands = df[['Dll1_2','1I4E_6.0','Dll4_6','4I1E_2.0']].copy()
df_ligands = df_ligands.rename(columns={'Dll1_2':'Dll1','1I4E_6.0':'1I4E','Dll4_6':'Dll4','4I1E_2.0':'4I1E'})

df_ligands


In [ ]:

fig, ax1 = plt.subplots()

sns.set(style="whitegrid")
tips = sns.load_dataset("tips")


sns.boxplot(data=df_ligands[["Dll1","4I1E"]], showfliers = False, ax = ax1, boxprops={'facecolor':'None'})
sns.swarmplot(data=df_ligands[["Dll1","4I1E"]], dodge = 0,size = 2.4, palette=[cm_ligands[0],cm_ligands[3]], ax = ax1, zorder=0)

ax1.set_xlabel(r'')
ax1.set_ylabel(r"Cit$_{max}$ [a.u.]")

ax1.set_ylim([200,400])

ax1.set_xticklabels([ligands[0],ligands[3]])

fig.savefig(figures_path + '/' + filename0 + '_H.png', bbox_inches="tight", dpi = 300, transparent = True)


In [ ]:

fig, ax1 = plt.subplots()

sns.set(style="whitegrid")
tips = sns.load_dataset("tips")


sns.boxplot(data=df_ligands[["Dll4","1I4E"]], showfliers = False, ax = ax1, boxprops={'facecolor':'None'})
sns.swarmplot(data=df_ligands[["Dll4","1I4E"]], dodge = 0,size = 2.2, palette=[cm_ligands[2],cm_ligands[1]], ax = ax1, zorder=0)

ax1.set_xlabel(r'')
ax1.set_ylabel(r"Cit$_{max}$ [a.u.]")

ax1.set_xticklabels([ligands[2],ligands[1]])

fig.savefig(figures_path + '/' + filename0 + '_J.png', bbox_inches="tight", dpi = 300, transparent = True)


In [ ]:
ppvv_11 = peak_values(T_H[0], sol[0].y[-1])
ppvv_14 = peak_values(T_H[1], sol[1].y[-1])
ppvv_44 = peak_values(T_H[2], sol[2].y[-1])
ppvv_41 = peak_values(T_H[3], sol[3].y[-1])

print("Measure ; Literature ; Simulation")
print("Dll1/4I1E ; " + str(round(c_max_11/c_max_41,2)) + " ; " + str(round(ppvv_11[2]/ppvv_41[2],2)))
print("1I4E/Dll4 ; " + str(round(c_max_14/c_max_44,2)) + " ; " + str(round(ppvv_14[2]/ppvv_44[2],2)))
print("FWHM Dll1 ; [12,13.3] h ; " + str(round(ppvv_11[1]-ppvv_11[0],2)) + ' h')
print("FWHM 1I4E ; ~15.63 h ; " + str(round(ppvv_14[1]-ppvv_14[0],2)) + ' h')
print("tmax Dll1 ; [5.7,7.1] h ; " + str(round(ppvv_11[-1],2)) + ' h')
print("tmax 1I4E ; ~6 h ; " + str(round(ppvv_14[-1],2)) + ' h')

# Panels E & F

In [ ]:
plt.style.use('default')
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams.update({'font.size': 13})

## Panel E

In [ ]:

params = dict(params_D1)

k_i = np.logspace(-2,3,200)

p_p_names = ["bl"]
p_p_values=[k_i]

p_p_values_iter = list(itertools.product(*p_p_values))
p_p_values_iter.sort(key=lambda a: a[0])


param_iter = []

for  e in p_p_values_iter:
    param_i = dict(params)
    param_i.update(zip(p_p_names, e))
    param_iter.append(tuple([dict(param_i)]))

x_0 = [[round(params["br"]/params["dr"],2), 
       round(e[0]/params["dl"],2),
       round(params["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in p_p_values_iter]


In [ ]:
for i in range(len(param_iter)):
    param_iter[i][0].update({'x0': x_0[i]})

partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)

# ncpus = None
ncpus = mp.cpu_count() - 2

def partial_iter_mp(param_iter):
    X_0 = param_iter[0]['x0']
    return partial_iter(X_0,'Radau',param_iter)

with mp.Pool(ncpus) as pool:
    sol = pool.map(partial_iter_mp, param_iter)


In [ ]:

cit_stat = [e.y[9][-1] for e in sol]
cit_stat_norm = [e/max(cit_stat) for e in cit_stat]

bl_12 = peak_values(k_i,cit_stat_norm)[4]
hill_1 = [bl / (bl_12 + bl) for bl in k_i]
hill_2 = [bl**2 / (bl_12 ** 2 + bl**2) for bl in k_i]
hill_3 = [bl**3 / (bl_12 ** 3 + bl**3) for bl in k_i]


In [ ]:

fig, ax= plt.subplots()

ligand = 0

ax.plot(k_i, cit_stat_norm, color = cm_ligands[ligand], label = ligands[ligand] , lw = 2.5)
ax.plot(k_i, hill_1, color = cm_ligands[ligand], ls = '-.', label = '$Hill_1$')
ax.plot(k_i, hill_2, color = cm_ligands[ligand], ls = ':', label = '$Hill_2$')
ax.plot(k_i, hill_3, color = cm_ligands[ligand], ls = '--', label = '$Hill_3$')

ax.legend()

ax.set_xscale('log')

ax.set_xlabel(r'$\beta_L\ (min^{-1})$', fontsize = 15)
ax.set_ylabel(r'Normalized $Cit_{Stat}$', fontsize = 15)

plt.show()

fig.savefig(figures_path + '/' + filename0 + '_E.png', bbox_inches="tight", dpi = 300, transparent = True)

## Panel F

In [ ]:

params = dict(params_D4)

k_i = np.logspace(-2,3,200)
p_p_names = ["bl"]
p_p_values=[k_i]

p_p_values_iter = list(itertools.product(*p_p_values))
p_p_values_iter.sort(key=lambda a: a[0])


param_iter = []

for  e in p_p_values_iter:
    param_i = dict(params)
    param_i.update(zip(p_p_names, e))
    param_iter.append(tuple([dict(param_i)]))

x_0 = [[round(params["br"]/params["dr"],2), 
       round(e[0]/params["dl"],2),
       round(params["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in p_p_values_iter]


In [ ]:
for i in range(len(param_iter)):
    param_iter[i][0].update({'x0': x_0[i]})

partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)

ncpus = None
# ncpus = mp.cpu_count() - 2

def partial_iter_mp(param_iter):
    X_0 = param_iter[0]['x0']
    return partial_iter(X_0,'Radau',param_iter)

with mp.Pool(ncpus) as pool:
    sol = pool.map(partial_iter_mp, param_iter)


In [ ]:

cit_stat = [e.y[9][-1] for e in sol]
cit_stat_norm = [e/max(cit_stat) for e in cit_stat]

bl_12 = peak_values(k_i,cit_stat_norm)[4]
hill_1 = [bl / (bl_12 + bl) for bl in k_i]
hill_2 = [bl**2 / (bl_12 ** 2 + bl**2) for bl in k_i]
hill_3 = [bl**3 / (bl_12 ** 3 + bl**3) for bl in k_i]


In [ ]:

fig, ax= plt.subplots()

ligand = 2

ax.plot(k_i, cit_stat_norm, color = cm_ligands[ligand], label = ligands[ligand] , lw = 2.5)
ax.plot(k_i, hill_1, color = cm_ligands[ligand], ls = '-.', label = '$Hill_1$')
ax.plot(k_i, hill_2, color = cm_ligands[ligand], ls = ':', label = '$Hill_2$')
ax.plot(k_i, hill_3, color = cm_ligands[ligand], ls = '--', label = '$Hill_3$')

ax.legend()

ax.set_xscale('log')

ax.set_xlabel(r'$\beta_L\ (min^{-1})$', fontsize = 15)
ax.set_ylabel(r'Normalized $Cit_{Stat}$', fontsize = 15)

plt.show()

fig.savefig(figures_path + '/' + filename0 + '_F.png', bbox_inches="tight", dpi = 300, transparent = True)

# Data for Table

In [ ]:

param_iter = []
list_dict_hybrids = []

params = dict(params_D1)
list_dict_hybrids.append(params)

params = dict(params_1I_4E)
list_dict_hybrids.append(params)

params = dict(params_D4)
list_dict_hybrids.append(params)

params = dict(params_4I_1E)
list_dict_hybrids.append(params)

for i in range(len(list_dict_hybrids)):
    params = dict(list_dict_hybrids[i])
    param_iter.append(tuple([dict(params)]))
    

# CI <> 0
x_0 = [[round(e[0]["br"]/e[0]["dr"],2), 
       round(e[0]["bl"]/e[0]["dl"],2),
       round(e[0]["E0"],2),
       0., 0., 0., 0., 0., 0., 0.] for e in param_iter]


In [ ]:
partial_iter = partial(par_solve_ivp, m_notch_epsin_coop, time_l)
sol = [partial_iter(f,'Radau',e) for e,f in zip(param_iter,x_0)]

T_H = [e.t/60. for e in sol]

In [ ]:
ppvv_11 = peak_values(T_H[0], sol[0].y[-1])
ppvv_14 = peak_values(T_H[1], sol[1].y[-1])
ppvv_44 = peak_values(T_H[2], sol[2].y[-1])
ppvv_41 = peak_values(T_H[3], sol[3].y[-1])

print("Measure ; Literature ; Simulation")
print("Dll1/4I1E ; " + str(round(c_max_11/c_max_41,2)) + " ; " + str(round(ppvv_11[2]/ppvv_41[2],2)))
print("1I4E/Dll4 ; " + str(round(c_max_14/c_max_44,2)) + " ; " + str(round(ppvv_14[2]/ppvv_44[2],2)))
print("FWHM Dll1 ; [12,13.3] h ; " + str(round(ppvv_11[1]-ppvv_11[0],2)) + ' h')
print("FWHM 1I4E ; ~15.63 h ; " + str(round(ppvv_14[1]-ppvv_14[0],2)) + ' h')
print("tmax Dll1 ; [5.7,7.1] h ; " + str(round(ppvv_11[-1],2)) + ' h')
print("tmax 1I4E ; ~6 h ; " + str(round(ppvv_14[-1],2)) + ' h')